In [1]:
from functools import partial

import cirq

from mitiq.calibration import Calibration, ZNESettings

from warnings import filterwarnings

filterwarnings("ignore", message="The input circuit is very short")

In [2]:
def execute(circuit, noise_level=0.001):
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

In [3]:
cal = Calibration(execute, ZNESettings, partial(execute, noise_level=0.0))
cal.get_cost()

{'noisy_executions': 3, 'ideal_executions': 3}

In [4]:
for c in cal.circuits:
    print(c.dimensions)
    print(c.type)
    print(c.two_qubit_gate_count)
    print(c.circuit)

(2, 2)
ghz
1
0: ───H───@───
          │
1: ───────X───
(2, 39)
rb
8
0: ───Y^0.5────X^0──────@───Y^0.5────X^0.5────Y^-0.5───X^0.5──────────@───Y^0.5────@───Y───────X^0.5───X────────Y^0.5───@───X^0───X^-0.5───Y^0──────────────@───Y^0.5────@───Y^0.5───Y───────X─────────────────────────────────@───Y^0.5────X^0.5────Y^-0.5───X^-0.5───────────────────@───Y^0.5───X^0.5───
                        │                                             │            │                                    │                                   │            │                                                     │                                                       │
1: ───Y^-0.5───X^-0.5───@───X^-0.5───Y^-0.5───X^0.5────Y^-0.5───X^0───@───X^-0.5───@───X^0.5───X^0.5───Y^-0.5───────────@───Y─────X^0.5────X^-0.5───Y^0.5───@───X^-0.5───@───X^0.5───Y^0.5───X^0.5───Y^-0.5───X^0.5───Y^-0.5───@───X^-0.5───Y^-0.5───X^0.5────Y^-0.5───X^0.5───Y^0.5───@───Y^0.5───────────
(2, 25)
mirror
10
0: ───X^-0.5───Y^-0.5───I───@───Z

In [5]:
cal.run_circuits()

In [6]:
cal.results # average mitigated value, best mitigated result

[{'unmitigated': 0.49900118,
  'mitigated': {'zne': {'results': [{'circuit_type': 'ghz',
      'extrapolation_method': 'RichardsonFactory',
      'scale_factors': [1.0, 2.0, 3.0],
      'scale_noise_method': 'fold_global',
      'mitigated_value': 0.5003295540809632}],
    'method_improvement_factor': None}},
  'ideal': 0.49999997},
 {'unmitigated': 0.937392,
  'mitigated': {'zne': {'results': [{'circuit_type': 'rb',
      'extrapolation_method': 'RichardsonFactory',
      'scale_factors': [1.0, 2.0, 3.0],
      'scale_noise_method': 'fold_global',
      'mitigated_value': 1.0229288339614862}],
    'method_improvement_factor': None}},
  'ideal': 1.0},
 {'unmitigated': 0.9610512,
  'mitigated': {'zne': {'results': [{'circuit_type': 'mirror',
      'extrapolation_method': 'RichardsonFactory',
      'scale_factors': [1.0, 2.0, 3.0],
      'scale_noise_method': 'fold_global',
      'mitigated_value': 1.0052748918533319}],
    'method_improvement_factor': None}},
  'ideal': 1.0}]

In [7]:
cal.compute_improvements()

In [10]:
cal.get_optimal_strategy()

|ideal - best| / ideal = 0.03967916965484686


'zne'

In [ ]:
from mitiq.benchmarks import generate_ghz_circuit, generate_rb_circuits

In [ ]:
ghz = generate_ghz_circuit(5)
rb = generate_rb_circuits(2, 10)[0]

In [ ]:
print(rb)

In [ ]:
rb.all_qubits()

In [ ]:
list(ghz.all_operations())[1].qubits

In [ ]:
len(list(filter(lambda op: len(op.qubits) > 1, ghz.all_operations())))

In [ ]:
sum([len(op.qubits) > 1 for op in ghz.all_operations()])